In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from newspaper import Article
import requests
import re
import datetime
from tqdm import tqdm
import sys

In [2]:
df = pd.read_csv("C:/동국대학교/공모전/미래에셋/뉴스데이터_최종.csv", encoding='utf-8')

In [3]:
df

,news_title,news_url,news_contents,news_dates
0,줍줍까지 나왔는데 웃돈만 10억…'얼죽신' 열풍에 입주권 귀하신 몸,https://n.news.naver.com/mnews/article/057/000...,[\n\t\t\t【 앵커멘트 】 '얼죽신'이라는 신조어가 있습니다. 얼어죽어도 ...,2024-07-26 20:06:12
1,‘AI 반도체’ 리벨리온 IPO 주관사에 대표 삼성·공동 한투 선정,https://n.news.naver.com/mnews/article/366/000...,[\n기업가치 5조~7조원 인공지능(AI) 반도체 스타트업 리벨리온이 ...,2024-07-26 17:46:14
2,서울 세운5-1·3구역 대출 연장 성공…규모 1000억 확대,https://n.news.naver.com/mnews/article/018/000...,[\n대출 2800억 조기 상환…신규 대출 3800억 확대SPC 투모로우세운·에이에...,2024-07-26 19:48:07
3,"배임까지… 두산밥캣, 6거래일째 하락 이어 시간外 약세",https://n.news.naver.com/mnews/article/366/000...,[\n\n\n\n\n두산밥캣의 스티어-스키드 로더. /두산밥캣 두산그룹...,2024-07-26 17:21:13
4,상승 출발 엔비디아 vs 하락 출발 테슬라 …과연 누가 웃을까?,https://n.news.naver.com/mnews/article/029/000...,[\n뉴욕증시 3대 지수 상승출발엔비디아 2%대 상승테슬라는 낙폭 줄여빅테크주 각자...,2024-07-26 22:56:07
...,...,...,...,...
922,"셀트리온, 하반기 짐펜트라 잠재력 확인-상상인",https://n.news.naver.com/mnews/article/014/000...,[\n\n\n\n\n [파이낸셜뉴스] 상상인증권은 22일 셀트리온에 대해 올해 하반...,2024-07-22 05:09:12
923,"열기 빠진 시장…피앤에스미캐닉스 VS 아이빔테크놀로지, 증권가 ‘픽’은",https://n.news.naver.com/mnews/article/018/000...,"[\n[IPO대IPO]재활로봇 기업, 생체현미경 제조사 이번주 릴레이 청약국내 1호...",2024-07-22 00:01:09
924,5대 금융 트래블 체크카드 가입자 650만명 육박,https://n.news.naver.com/mnews/article/001/001...,[\n휴가철 앞두고 고객 유치전 치열…신용카드로 경쟁 확대\n\n\n\n[연합뉴스 ...,2024-07-22 06:15:01
925,금융당국 채찍 드니까 PF 손실 인식… 대형 증권사도 2분기 어닝쇼크 가능성 커져,https://n.news.naver.com/mnews/article/366/000...,[\n6월부터 PF 사업성 평가 기준 바뀌어빡빡해진 기준에 늘어날 부실 사업장2분기...,2024-07-22 06:01:22


In [4]:
def preprocess_text(text):
    # 불필요한 공백 및 개행 문자 제거
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    # 불필요한 특수 문자 제거
    text = re.sub(r'\[사진 출처.*?\]', '', text)
    text = re.sub(r'^\[|\]$', '', text)
    return text


In [5]:
# 'news_contents' 열 전처리
df['cleaned_news_contents'] = df['news_contents'].apply(preprocess_text)

print(df['cleaned_news_contents'][0])

 【 앵커멘트 】 '얼죽신'이라는 신조어가 있습니다. 얼어죽어도 신축이라는 새 아파트 선호현상을 가리키는 표현이죠. 공급 위축 우려에 30대를 중심으로 이런 현상이 강해지다 보니, 새 아파트 입주권은 가격이 치솟고 있습니다. 이승훈 기자입니다.【 기자 】 오는 11월 입주를 앞둔 서울 강동구의 한 재건축 아파트입니다. 1만2천 가구, 우리나라 최대 단지임에도 작년 미분양이 나 무순위청약, 이른바 줍줍까지 진행됐지만, 지금은 180도 분위기가 달라졌습니다. 전용면적 84㎡ 입주권에는 웃돈이 10억 원 가량 붙었습니다.▶ 인터뷰 : 공인중개사- "(전용면적 84㎡) 24억 원에도 거래됐다고 하더라고. 지금은 로열층도 타입도 중요하지 않더라고. 25억에도 물건이 안 잡히니까." 서울 동작구의 이 아파트도 작년 고분양가 논란으로 200가구 가량 미분양됐지만, 최근 완판을 앞두고 있습니다. 공사비 상승으로 공급이 줄고 분양가가 뛰자, 재개발재건축 단지 입주권과 미분양 신축아파트로 수요가 몰린 겁니다.▶ 스탠딩 : 이승훈 / 기자- "특히 젊은 세대를 중심으로 신축을 선호하는 모습인데, 얼어 죽어도 신축, 이른바 '얼죽신'이라는 신조어까지 등장할 정도입니다." 이번 달 수도권 1순위 청약 경쟁률도 평균 100대 1에 육박해 역대 두번째로 높은 수준까지 치솟았습니다.▶ 인터뷰(☎) : 권대중 / 서강대 부동산대학원 교수- "향후 주택 공급이 부족하다는 시그널 때문에 가격이 오른 것 같습니다. 현재 아파트 쏠림 현상을 비 아파트 부분도 활성화될 수 있는 정책이 좀 더 필요한…." 서울 집값이 이상과열 양상을 보이자 정부는 다음 달 주택공급을 획기적으로 늘리는 방안을 발표하기로 했습니다. MBN뉴스 이승훈입니다.영상취재 : 문진웅 기자 영상편집 : 이주호 그래픽 : 최지훈 


In [6]:
df['cleaned_news_contents']

0       【 앵커멘트 】 '얼죽신'이라는 신조어가 있습니다. 얼어죽어도 신축이라는 새 아파...
1       기업가치 5조~7조원 인공지능(AI) 반도체 스타트업 리벨리온이 기업공개(IPO)...
2       대출 2800억 조기 상환…신규 대출 3800억 확대SPC 투모로우세운·에이에이엠...
3       두산밥캣의 스티어-스키드 로더. /두산밥캣 두산그룹 지배구조 개편 여파로 두산밥캣...
4       뉴욕증시 3대 지수 상승출발엔비디아 2%대 상승테슬라는 낙폭 줄여빅테크주 각자도생...
                             ...                        
922     [파이낸셜뉴스] 상상인증권은 22일 셀트리온에 대해 올해 하반기는 짐펜트라의 잠재...
923     [IPO대IPO]재활로봇 기업, 생체현미경 제조사 이번주 릴레이 청약국내 1호 보...
924     휴가철 앞두고 고객 유치전 치열…신용카드로 경쟁 확대 [연합뉴스 자료사진] (서울...
925     6월부터 PF 사업성 평가 기준 바뀌어빡빡해진 기준에 늘어날 부실 사업장2분기 실...
926     3년물 이달 15거래일 연속 순매수…누적 8만8천계약"아시아 주요국 중 가장 먼저...
Name: cleaned_news_contents, Length: 927, dtype: object

In [7]:
df

,news_title,news_url,news_contents,news_dates,cleaned_news_contents
0,줍줍까지 나왔는데 웃돈만 10억…'얼죽신' 열풍에 입주권 귀하신 몸,https://n.news.naver.com/mnews/article/057/000...,[\n\t\t\t【 앵커멘트 】 '얼죽신'이라는 신조어가 있습니다. 얼어죽어도 ...,2024-07-26 20:06:12,【 앵커멘트 】 '얼죽신'이라는 신조어가 있습니다. 얼어죽어도 신축이라는 새 아파...
1,‘AI 반도체’ 리벨리온 IPO 주관사에 대표 삼성·공동 한투 선정,https://n.news.naver.com/mnews/article/366/000...,[\n기업가치 5조~7조원 인공지능(AI) 반도체 스타트업 리벨리온이 ...,2024-07-26 17:46:14,기업가치 5조~7조원 인공지능(AI) 반도체 스타트업 리벨리온이 기업공개(IPO)...
2,서울 세운5-1·3구역 대출 연장 성공…규모 1000억 확대,https://n.news.naver.com/mnews/article/018/000...,[\n대출 2800억 조기 상환…신규 대출 3800억 확대SPC 투모로우세운·에이에...,2024-07-26 19:48:07,대출 2800억 조기 상환…신규 대출 3800억 확대SPC 투모로우세운·에이에이엠...
3,"배임까지… 두산밥캣, 6거래일째 하락 이어 시간外 약세",https://n.news.naver.com/mnews/article/366/000...,[\n\n\n\n\n두산밥캣의 스티어-스키드 로더. /두산밥캣 두산그룹...,2024-07-26 17:21:13,두산밥캣의 스티어-스키드 로더. /두산밥캣 두산그룹 지배구조 개편 여파로 두산밥캣...
4,상승 출발 엔비디아 vs 하락 출발 테슬라 …과연 누가 웃을까?,https://n.news.naver.com/mnews/article/029/000...,[\n뉴욕증시 3대 지수 상승출발엔비디아 2%대 상승테슬라는 낙폭 줄여빅테크주 각자...,2024-07-26 22:56:07,뉴욕증시 3대 지수 상승출발엔비디아 2%대 상승테슬라는 낙폭 줄여빅테크주 각자도생...
...,...,...,...,...,...
922,"셀트리온, 하반기 짐펜트라 잠재력 확인-상상인",https://n.news.naver.com/mnews/article/014/000...,[\n\n\n\n\n [파이낸셜뉴스] 상상인증권은 22일 셀트리온에 대해 올해 하반...,2024-07-22 05:09:12,[파이낸셜뉴스] 상상인증권은 22일 셀트리온에 대해 올해 하반기는 짐펜트라의 잠재...
923,"열기 빠진 시장…피앤에스미캐닉스 VS 아이빔테크놀로지, 증권가 ‘픽’은",https://n.news.naver.com/mnews/article/018/000...,"[\n[IPO대IPO]재활로봇 기업, 생체현미경 제조사 이번주 릴레이 청약국내 1호...",2024-07-22 00:01:09,"[IPO대IPO]재활로봇 기업, 생체현미경 제조사 이번주 릴레이 청약국내 1호 보..."
924,5대 금융 트래블 체크카드 가입자 650만명 육박,https://n.news.naver.com/mnews/article/001/001...,[\n휴가철 앞두고 고객 유치전 치열…신용카드로 경쟁 확대\n\n\n\n[연합뉴스 ...,2024-07-22 06:15:01,휴가철 앞두고 고객 유치전 치열…신용카드로 경쟁 확대 [연합뉴스 자료사진] (서울...
925,금융당국 채찍 드니까 PF 손실 인식… 대형 증권사도 2분기 어닝쇼크 가능성 커져,https://n.news.naver.com/mnews/article/366/000...,[\n6월부터 PF 사업성 평가 기준 바뀌어빡빡해진 기준에 늘어날 부실 사업장2분기...,2024-07-22 06:01:22,6월부터 PF 사업성 평가 기준 바뀌어빡빡해진 기준에 늘어날 부실 사업장2분기 실...


In [8]:
df.to_csv("C:/동국대학교/공모전/미래에셋/뉴스데이터_최종_cleaned.csv", encoding='utf-8', index=False)